In [1]:
from processing import load_image_dataset

x_rgb_train, y_rgb_train = load_image_dataset('Datasets/espe/lbp_histograms - copia/train', (40, 40), True)
x_rgb_test, y_rgb_test = load_image_dataset('Datasets/espe/lbp_histograms - copia/test', (40, 40), True)
x_rgb_validation, y_rgb_validation = load_image_dataset('Datasets/espe/lbp_histograms - copia/validation', (40, 40), True)

x_masks_train, y_masks_train = load_image_dataset('Datasets/espe/masks/train', (40, 40), True)
x_masks_test, y_masks_test = load_image_dataset('Datasets/espe/masks/test', (40, 40), True)
x_masks_validation, y_masks_validation = load_image_dataset('Datasets/espe/masks/validation', (40, 40), True)

Using TensorFlow backend.


In [2]:
import numpy as np
x_masks, y_masks = np.concatenate((x_masks_train, x_masks_validation), axis=0), np.concatenate((y_masks_train, y_masks_validation), axis=0)
x_rgb, y_rgb = np.concatenate((x_rgb_train/255, x_rgb_validation/255), axis=0), np.concatenate((y_rgb_train, y_rgb_validation), axis=0)

In [3]:
x_masks.shape == x_rgb.shape, y_masks.shape == y_rgb.shape

(True, True)

In [11]:
x_data = [x_rgb, x_masks]
x_data.shape

AttributeError: 'list' object has no attribute 'shape'

In [4]:
from training.utils import cv_training
cv_training(x_data=[x_rgb, x_masks], y_data=[y_rgb])

TypeError: list indices must be integers or slices, not tuple

In [17]:
from sklearn.model_selection import KFold
from utils.Models import *
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

loss_per_fold = []
acc_per_fold = []

kf = KFold(n_splits=10, shuffle=True)
fold_no = 1
for train_index, test_index in kf.split(x_masks):
    # load model
    model = prototype_model_for_reid((40, 40 , 1))

    # creating callbacks
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=8)
    reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=4, verbose=1, min_delta=1e-3)
    callbacks_list = [stop_early, reduce_lr]

    # fitting model
    history = model.fit(x=[x_rgb[train_index], x_masks[train_index]], y=y_rgb[train_index], batch_size=150, epochs=20, validation_split=0.2)

    # saving model
    model.save(f'models/own/experiments/experiment_4/sample_model{fold_no}.h5')

    # plot training metrics
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model accuracy')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.savefig(f'acc_{fold_no}.png')
    plt.clf()

    # generate generalization metrics
    scores = model.evaluate(x=[x_rgb[test_index], x_masks[test_index]], y=y_rgb[test_index], verbose=True)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1]*100)
    loss_per_fold.append(scores[0])

    fold_no += 1

Train on 9384 samples, validate on 2346 samples
Epoch 1/20
9384/9384 [==============================] - 5s 523us/sample - loss: 27.7096 - acc: 0.1725 - val_loss: 19.7031 - val_acc: 0.1624
Epoch 2/20
9384/9384 [==============================] - 3s 275us/sample - loss: 4.9944 - acc: 0.2239 - val_loss: 2.2705 - val_acc: 0.2123
Epoch 3/20
9384/9384 [==============================] - 3s 275us/sample - loss: 1.8618 - acc: 0.3529 - val_loss: 2.6945 - val_acc: 0.2298
Epoch 4/20
9384/9384 [==============================] - 3s 276us/sample - loss: 1.2486 - acc: 0.6012 - val_loss: 1.5411 - val_acc: 0.4885
Epoch 5/20
9384/9384 [==============================] - 3s 279us/sample - loss: 0.7226 - acc: 0.7491 - val_loss: 1.5547 - val_acc: 0.5175
Epoch 6/20
9384/9384 [==============================] - 3s 279us/sample - loss: 0.4200 - acc: 0.8638 - val_loss: 1.6791 - val_acc: 0.5563
Epoch 7/20
9384/9384 [==============================] - 3s 301us/sample - loss: 0.2763 - acc: 0.9102 - val_loss: 2.2851 - 

<Figure size 432x288 with 0 Axes>

In [18]:
from tensorflow.keras.models import load_model
trained_model = load_model('models/own/saved_model/sample_model3.h5')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [55]:
result = trained_model.predict(x=[[x_rgb_test[2000]/255], [x_masks_test[2000]]])
result.argmax()

2

array([[1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [75]:
model.evaluate(x=[[x_rgb_test[0]/255], [x_masks_test[0]]], y=y_masks_test[:1, :])


1/1 [==============================] - 0s 161ms/sample - loss: 0.6636 - acc: 1.0000


[0.6635879278182983, 1.0]

In [83]:
print(x_rgb_test[0][0][0]/255)

[0.09411765]


In [8]:
from tensorflow.keras.models import load_model
trained_model_2 = load_model('models/own/experiments/experiment_3/sample_model3.h5')
trained_model_2.predict(x=[[x_rgb_test[0]], [x_masks_test[0]]])

array([[6.4668113e-01, 5.5572696e-07, 6.2391174e-04, 4.2988219e-08,
        1.9188530e-05, 3.5267511e-01, 6.5811783e-08]], dtype=float32)